# Discovering Earth Observation Data with the EO-MQS

The EO-MQS service is hosted within the C-SCALE federated cloud infrastructure and provides a unified way of discovering Copernicus data available within the federation by making use of the SpatioTemporal Asset Catalog (STAC) specification. The purpose of this notebook is to prvovide a concise introduction on how to use open-source Python libraries to search for geospatial data exposed by the EO-MQS STAC API.

## Prerequisites

In this example, we are going to make use of a popular STAC client for Python, the `pystac-client`. The library is already installed in this environment, but can be manually installed anywhere else via `pip install pystac-client`. 
Alternatively, common Python libraries like the `requests` library which support working with HTTP APIs are of course also well suited.

To get started, we need to import the `Client` class to connect to the EO-MQS which exposes its STAC API under `https://mqs.eodc.eu/stac/v1`.

In [1]:
from pystac_client import Client

client = Client.open("https://mqs.eodc.eu/stac/v1")


In [2]:
client.title

'C-SCALE Earth Observation Metadata Query Service (EO-MQS)'

## CollectionClient

The client can be used to iterate through the Collections available in the EO-MQS Catalog. 

The `get_collections` method fetches the collections from the `/collections` endpoint and returns an iterable. To load a particular collection for further use we call the `get_collection(<collection_id>)` method below.

In [3]:
for collection in client.get_collections():
    print(collection)

<CollectionClient id=EODC|sentinel1-grd>
<CollectionClient id=EODC|sentinel-2-l1c>
<CollectionClient id=EODC|s1-global-sigma0>
<CollectionClient id=EODC|s1-demo-sigma0>
<CollectionClient id=GRNET-OPENSTACK|sentinel-1-grd>
<CollectionClient id=GRNET-OPENSTACK|sentinel-1-ocn>
<CollectionClient id=GRNET-OPENSTACK|sentinel-1-raw>
<CollectionClient id=GRNET-OPENSTACK|sentinel-1-slc>
<CollectionClient id=GRNET-OPENSTACK|sentinel-2-l1b>
<CollectionClient id=GRNET-OPENSTACK|sentinel-2-l1c>
<CollectionClient id=GRNET-OPENSTACK|sentinel-2-l2a>
<CollectionClient id=GRNET-OPENSTACK|sentinel-3-olci-l1b>
<CollectionClient id=GRNET-OPENSTACK|sentinel-3-olci-l2>
<CollectionClient id=GRNET-OPENSTACK|sentinel-3-slstr-l1b>
<CollectionClient id=GRNET-OPENSTACK|sentinel-3-slstr-l2>
<CollectionClient id=GRNET-OPENSTACK|sentinel-3-stm-l2>
<CollectionClient id=GRNET-OPENSTACK|sentinel-3-syn-l2>
<CollectionClient id=GRNET-OPENSTACK|sentinel-5p-l1b>
<CollectionClient id=GRNET-OPENSTACK|sentinel-5p-l2>
<Collecti

On static as well as dynamic catalogues we cann also make use of the `links` attributes which lets us quickly examinate, for instance, the number of available collections.

In [4]:
child_links = client.get_links('child')
print(f"The EO-MQS currently features {len(child_links)} collections.")

The EO-MQS currently features 94 collections.


In [5]:
collection = client.get_collection("EODC|sentinel-2-l1c")
#collection = client.get_collection("sentinel-2-l1c")
collection

id: EODC|sentinel-2-l1c
title: Sentinel-2 MSI Products: Level-1C data
"description: SENTINEL-2 is a wide-swath, high-resolution, multi-spectral imaging mission, supporting Copernicus Land Monitoring studies, including the monitoring of vegetation, soil and water cover, as well as observation of inland waterways and coastal areas. The SENTINEL-2 Multispectral Instrument (MSI) samples 13 spectral bands: four bands at 10 metres, six bands at 20 metres and three bands at 60 metres spatial resolution. The acquired data, mission coverage and high revisit frequency provides for the generation of geoinformation at local, regional, national and international scales. The data is designed to be modified and adapted by users interested in thematic areas such as: - spatial planning - agro-environmental monitoring - water monitoring - forest and vegetation monitoring - land carbon, - natural resource monitoring - global crop monitoring. [https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-2-msi/overview](https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-2-msi/overview) The Level-1C product is composed of 100x100 km2 tiles (ortho-images in UTM/WGS84 projection). The Level-1C product results from using a Digital Elevation Model (DEM) to project the image in cartographic geometry. Per-pixel radiometric measurements are provided in Top Of Atmosphere (TOA) reflectances along with the parameters to transform them into radiances. Level-1C products are resampled with a constant Ground Sampling Distance (GSD) of 10, 20 and 60 m depending on the native resolution of the different spectral bands. In Level-1C products, pixel coordinates refer to the upper left corner of the pixel. Level-1C products will additionally include Cloud Masks and ECMWF data (total column of ozone, total column of water vapour and mean sea level pressure). [https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-2-msi/product-types/level-1c](https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-2-msi/product-types/level-1c)."
"providers: ESA (producer, processor, licensor) EODC (host)"
type: Collection
https://stac-extensions.github.io/sat/v1.0.0/schema.json
https://stac-extensions.github.io/eo/v1.0.0/schema.json
https://stac-extensions.github.io/projection/v1.0.0/schema.json
https://stac-extensions.github.io/alternate-assets/v1.1.0/schema.json
id: S2B_MSIL1C_20221110T000219_R030_T56JKS_20221110T005805
"bbox: [150.74103773087342, -26.295042324224823, 151.11048990713982, -25.30236686976029]"


There are many ways to access the collection metadata programmatically. 

In [6]:
print(f"This collection contains data in the following temporal inteval: {collection.extent.temporal.to_dict()}")

This collection contains data in the following temporal inteval: {'interval': [['2015-07-04T00:00:00Z', None]]}


In [7]:
# To verify this extent, we can calculate the actual limits like this:
collection.update_extent_from_items()

In [8]:
collection.extent.temporal.to_dict()

{'interval': [['2018-01-02T10:24:21.026000Z', '2022-11-10T00:02:19.024000Z']]}

In [9]:
# Check which STAC Extensions are used by the collection
collection.stac_extensions

['https://stac-extensions.github.io/sat/v1.0.0/schema.json',
 'https://stac-extensions.github.io/eo/v1.0.0/schema.json',
 'https://stac-extensions.github.io/projection/v1.0.0/schema.json',
 'https://stac-extensions.github.io/alternate-assets/v1.1.0/schema.json']

## STAC Items

Simlarly to before, we can use the collection client instance to iterate over the items contained in the collection. The server must provide the `/collections/<collection_id>/items` endpoint to support this feature automatically. This can be useful to manually filter items or extract information programmatically. The `get_all_items()` method again returns an iterator.

In [10]:
items = collection.get_all_items()

In [11]:
# Load 10 items with cloud cover less than 10%
items10 = []
for n, item in enumerate(items):
    if len(items10) == 10:
        break
    cloud_cover = item.properties.get("eo:cloud_cover")
    if cloud_cover < 10:
        print(f"Append item {item.id} with {cloud_cover:.2f}% cloud cover")
        items10.append(item)

Append item S2B_MSIL1C_20221110T000119_R030_T56KRF_20221110T005746 with 0.00% cloud cover
Append item S2B_MSIL1C_20221110T000119_R030_T56KQF_20221110T005746 with 0.25% cloud cover
Append item S2B_MSIL1C_20221110T000119_R030_T56KPG_20221110T005746 with 1.31% cloud cover
Append item S2B_MSIL1C_20221110T000119_R030_T56KPF_20221110T005746 with 6.87% cloud cover
Append item S2B_MSIL1C_20221110T000119_R030_T56KNF_20221110T005746 with 8.60% cloud cover
Append item S2A_MSIL1C_20220101T052231_R062_T44RLS_20220101T072023 with 1.74% cloud cover
Append item S2A_MSIL1C_20220101T052231_R062_T44RLR_20220101T072023 with 1.50% cloud cover
Append item S2A_MSIL1C_20220101T052231_R062_T44RKT_20220101T072023 with 2.51% cloud cover
Append item S2A_MSIL1C_20220101T052231_R062_T44RKS_20220101T072023 with 0.60% cloud cover
Append item S2A_MSIL1C_20220101T052231_R062_T44RKR_20220101T072023 with 9.30% cloud cover


In [12]:
items10[-1]

id: S2A_MSIL1C_20220101T052231_R062_T44RKR_20220101T072023
"bbox: [77.94966291515033, 27.001706217086564, 79.08260664643534, 28.011816421914702]"
datetime: 2022-01-01T05:22:31.024000Z
platform: Sentinel-2A
proj:epsg: 32644
"providers: [{'url': 'https://earth.esa.int/web/guest/home', 'name': 'ESA', 'roles': ['producer', 'processor', 'licensor']}]"
instruments: ['msi']
s2:mgrs_tile: 44RKR
constellation: Sentinel 2
s2:granule_id: S2A_OPER_MSI_L1C_TL_VGS1_20220101T072023_A034093_T44RKR_N03.01
eo:cloud_cover: 9.2964


In [13]:
# If the item provides a previeww image we can look at it in here using the following code
from IPython.display import Image

Image(url=items10[-1].assets["thumbnail-png"].href, width=500)

## Item Search

Data providers that have realized their STAC implementation in terms of a dynamic STAC API offer users the opportunity to search their Catalogs using spatial and temporal constraints. The `pystac_client` enables this search via the class method `search`. This function returns an ItemSearch instance that can further be accessed to retrieve matched items.

Note that in its current implementation, the EO-MQS supports the *core* STAC search endpoint paramters as described in the [STAC API - Item Search](https://github.com/radiantearth/stac-api-spec/tree/master/item-search#query-parameter-table) specification. Those are:
- limit
- bbox
- datetime
- intersects
- ids
- collections


### Example 1: Search for Sentinel-1 GRD data over Austria (bbox) in 2022

This first example makes use of the `bbox`, `datetime` and the `collections` parameters. Learn about the correct formatting of these values on the STAC Spec GitHub page or by looking at the [pystac-client docs](https://pystac-client.readthedocs.io/en/latest/api.html#item-search).

In [14]:
# We can iterate and grep the collections automatically or look it up in the browser or API
s1_collections = []
for collection in client.get_collections():
    if "grd" in collection.id.lower() :
        print(f"Append collection {collection.id} to list of Sentinel-1 collections.")
        s1_collections.append(collection.id)

# manually add collections as requried
s1_collections.append("CREODIAS|SENTINEL-1")

Append collection EODC|sentinel1-grd to list of Sentinel-1 collections.
Append collection GRNET-OPENSTACK|sentinel-1-grd to list of Sentinel-1 collections.
Append collection CollGS_CZ|sentinel-1-grd to list of Sentinel-1 collections.
Append collection VITO|urn:eop:VITO:CGS_S1_GRD_L1 to list of Sentinel-1 collections.
Append collection VITO|urn:eop:VITO:CGS_S1_GRD_SIGMA0_L1 to list of Sentinel-1 collections.


If you do not have bbox coordinates at hand, you can quickly create your region of interest at [geojson.io](https://geojson.io).

In [15]:
bbox_aut = [9.25, 46.31, 17.46, 49.18]
time_period = "2022-01-01/2022-10-31"
limit = 20 # limit the number of items to be returned (per data provider)

In [16]:
# put together the search dictionary
search1 = {"collections": s1_collections,
           "bbox": bbox_aut,
           "datetime": time_period,
           "limit": limit}    

In [17]:
results1 = client.search(**search1)

In [18]:
items = results1.item_collection()

print(f"We found {len(items)} matching items.")

We found 3 matching items.


#### Tips on how to increase match rate

**NOTE:** Depending on the backend implementation, the `collections` parameter might have restrictions on allowed values. To make sure all possible items are fetched, consider iterating over the collections in our list and issue separate requests.

In [19]:
items_list = []
for s1_collection in s1_collections:
    results = client.search(collections=[s1_collection], 
                            bbox=bbox_aut, 
                            datetime=time_period, 
                            limit=limit)
    try:
        items_list.extend(results.item_collection())
    except:
        print(f"Search for items with collection id {s1_collection} failed or no items found.")


Search for items with collection id GRNET-OPENSTACK|sentinel-1-grd failed or no items found.
Search for items with collection id CollGS_CZ|sentinel-1-grd failed or no items found.
Search for items with collection id CREODIAS|SENTINEL-1 failed or no items found.


In [20]:
print(f"Now, we found {len(items_list)} matching items.")

Now, we found 43 matching items.


**NOTE:** By default, `pystac_client.search` will choose the HTTP method *POST* when making requests to the STAC API. Some data providers will only allow *GET* requests! Potentially available datasets might therefore remain undetected. *Hint: specify the method explicitly.*

In [21]:
items_list = []
for s1_collection in s1_collections:
    results = client.search(collections=[s1_collection], 
                            bbox=bbox_aut, 
                            datetime=time_period, 
                            limit=limit,
                            method="GET")
    try:
        items_list.extend(results.item_collection())
    except:
        print(f"Search for items with collection id {s1_collection} failed.")


Search for items with collection id GRNET-OPENSTACK|sentinel-1-grd failed.
Search for items with collection id CREODIAS|SENTINEL-1 failed.


In [22]:
print(f"Now, we found {len(items_list)} matching items.")

Now, we found 63 matching items.


In [23]:
print(items_list[0].id)
Image(url=items_list[0].assets["thumbnail"].href, width=500)

S1A_IW_GRDH_1SDV_20220101T164242_20220101T164307_041270_04E7B4_21E3


### Example 2: Search for Sentinel-2 data intersecting a GeoJSON object

The second example makes use of the `intersects` and the `collections` parameters. Note that you cannot specify both `bbox` and `intersects`, this will result in an error.

We make use of a geojson file located in the data folder.

In [24]:
import json

with open('../data/portugal_spain.geojson') as f:
    geom = json.load(f)

In [25]:
geom

{'coordinates': [[[-9.806360451464798, 43.30814861718568],
   [-9.174474713396421, 42.283523999613124],
   [-9.231345901554931, 40.41134287072376],
   [-10.216107372265327, 38.66893463340165],
   [-9.224511698762683, 37.868653108645844],
   [-9.311934888643805, 36.830999575117374],
   [-8.446363194979341, 36.753838992859215],
   [-7.139289473112058, 36.72378951340109],
   [-6.79325028632249, 36.15808850115742],
   [-5.729284132254691, 35.801661091482714],
   [-4.792065473035137, 36.47414834822082],
   [-3.268692080987819, 36.450645615913984],
   [-0.865088958138756, 37.11001478604629],
   [0.40445312693046276, 38.54779188867866],
   [0.14411045039250325, 39.254403576749155],
   [0.6650689048820482, 40.13216065231481],
   [3.2163254439648767, 41.4960613087793],
   [3.582468779268595, 42.395825866288476],
   [0.47473299914148015, 43.035317751001855],
   [-3.5212823427808075, 43.6803777743371],
   [-8.14735945119611, 44.117211711625686],
   [-9.806360451464798, 43.30814861718568]]],
 'typ

In [26]:
s2_collections = []
for collection in client.get_collections():
    if "l1c" in collection.id.lower() :
        print(f"Append collection {collection.id} to list of Sentinel-2 L1C collections.")
        s2_collections.append(collection.id)


Append collection EODC|sentinel-2-l1c to list of Sentinel-2 L1C collections.
Append collection GRNET-OPENSTACK|sentinel-2-l1c to list of Sentinel-2 L1C collections.
Append collection CollGS_CZ|sentinel-2-l1c to list of Sentinel-2 L1C collections.
Append collection VITO|urn:eop:VITO:CGS_S2_L1C to list of Sentinel-2 L1C collections.


We pick one that supports POST requests, for instance *VITO|urn:eop:VITO:CGS_S2_L1C*.

In [27]:
search2 = {"collections": ["VITO|urn:eop:VITO:CGS_S2_L1C"],
           "intersects": geom,
           "limit": limit,
           "method": "POST"}    

In [28]:
results2 = client.search(**search2)

In [29]:
items = results2.item_collection()

print(f"We found {len(items)} matching items.")

We found 20 matching items.


In [30]:
items[0]

id: urn:eop:VITO:CGS_S2_L1C:S2A_MSIL1C_20161004T110912_N0204_R137_T30SVH_20161004T110942
"bbox: [-4.1527514, 38.1791242, -3.9144534, 38.8443712]"
datetime: 2016-10-04T11:09:12.026Z
title: S2A_MSIL1C_20161004T110912_N0204_R137_T30SVH_20161004T110942
created: 2022-10-12T03:33:57Z
updated: 2022-10-12T03:33:41.720Z
start_datetime: 2016-10-04T11:09:12.026Z
end_datetime: 2016-10-04T11:09:12.026Z
platform: Sentinel-2
href: https://services.terrascope.be/download/CGS_S2_L1C/2016/10/04/S2A_MSIL1C_20161004T110912_N0204_R137_T30SVH_20161004T110942/S2A_MSIL1C_20161004T110912_N0204_R137_T30SVH_20161004T110942.SAFE/GRANULE/L1C_T30SVH_A006712_20161004T110942/QI_DATA/T30SVH_20161004T110912_PVI.jp2
type: image/jp2


NOTE: You can always visualize STAC data (collections, items, etc.) in external tools like the STAC Browser, for instance do the following:

In [31]:
print(f"Look at this item in the STAC Browser: https://radiantearth.github.io/stac-browser/#/external/{items[0].get_self_href()}")

Look at this item in the STAC Browser: https://radiantearth.github.io/stac-browser/#/external/https://mqs.eodc.eu/stac/v1/collections/VITO|urn:eop:VITO:CGS_S2_L1C/items/urn:eop:VITO:CGS_S2_L1C:S2A_MSIL1C_20161004T110912_N0204_R137_T30SVH_20161004T110942


## Example 3: Use non-default search parameters (experimental!)

As mentioned, the EO-MQS officially does not support parameters that are not part of the STAC API core specifications. However, when realizing a STAC implementation at a data provider's site, we usually make use of open-source libraries that are constantly being developed and improved. An example for such an improvement is the addition of search paramters like `filter`, `sortby` or `fields`.

This section hints at what can be done using these additional parameters.

In [32]:
# let's re-use a sentinel-2 collection from before
search3 = {"collections": s2_collections[0],
           "limit": limit,
           "method": "GET"}    

In [33]:
# sort the results in a descending manner based on the datetime property
sortby = "-properties.datetime"
search_sort = search3
search_sort["sortby"] = sortby

In [34]:
results3 = client.search(**search_sort)
items = results3.item_collection()

In [35]:
print(f"First item {items[0]} has datetime {items[0].get_datetime()}")
print(f"Lastt item {items[-1]} has datetime {items[-1].get_datetime()}")

First item <Item id=S2B_MSIL1C_20221110T000219_R030_T56JKS_20221110T005805> has datetime 2022-11-10 00:02:19.024000+00:00
Lastt item <Item id=S2A_MSIL1C_20220101T052231_R062_T44RLN_20220101T072023> has datetime 2022-01-01 05:22:31.024000+00:00


In [36]:
# filter based on specific property, e.g. cloud cover
filterby = {
  "filter": {
    "op": "and",
    "args": [
      {
        "op": "<",
        "args": [
          {
            "property": "eo:cloud_cover"
          },
          10
        ]
      }
    ]
  }
}
search_filter = search3
search_filter["sortby"] = "-properties.eo:cloud_cover"
search_filter["filter"] = filterby
search_filter["method"] = "POST"
search_filter["limit"] = 100

In [37]:
results3 = client.search(**search_filter)
items = results3.item_collection()

In [38]:
print(f"First item {items[0]} has cloud cover {items[0].properties.get('eo:cloud_cover')}%")
print(f"Last item {items[-1]} has cloud cover {items[-1].properties.get('eo:cloud_cover')}%")


First item <Item id=S2A_MSIL1C_20180919T102021_R065_T33VXG_20180919T123234> has cloud cover 9.9377%
Last item <Item id=S2B_MSIL1C_20220101T012709_R131_T52KED_20220101T022936> has cloud cover 6.0682%
